## Deep Learning for Human Activity Recognition - Basic Data Analysis

In [2]:
# Suppress warnings 
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

### 1. Imports

In [3]:
import numpy as np
import pandas as pd
np.set_printoptions(linewidth=200)
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 100

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="darkgrid")

# import other libraries
import os, pickle, json
from collections import Counter 
from operator import itemgetter

### 2. Read in Data

In [4]:
root = '../data/hapt_data_set/'
print(os.listdir(root))
print(os.listdir(root + 'Train'))
print(os.listdir(root + 'Test'))

['Test', 'features.txt', 'activity_labels.txt', 'features_info.txt', 'Train', 'README.txt', 'RawData']
['X_train.txt', 'subject_id_train.txt', 'y_train.txt']
['X_test.txt', 'y_test.txt', 'subject_id_test.txt']


In [5]:
features = pd.read_table(root + 'features.txt', header=None).values.flatten()
features = np.array([feature.rstrip() for feature in features])

In [6]:
%%time
X_train = pd.read_table(root + 'Train/X_train.txt', sep=' ', header=None, names=features)
y_train = pd.read_table(root + 'Train/y_train.txt', sep=' ', header=None)
subject_id_train = pd.read_table(root + 'Train/subject_id_train.txt', sep=' ', header=None)

X_test = pd.read_table(root + 'Test/X_test.txt', sep=' ', header=None, names=features)
y_test = pd.read_table(root + 'Test/y_test.txt', sep=' ', header=None)
subject_id_test = pd.read_table(root + 'Test/subject_id_test.txt', sep=' ', header=None)

CPU times: user 1.37 s, sys: 138 ms, total: 1.51 s
Wall time: 1.73 s


### 3. Glimpse of Data


In [7]:
print('Size of train data', X_train.shape)
display(pd.DataFrame(X_train.dtypes, columns=['dtype']).T)
display(X_train.head(3))

Size of train data (7767, 561)


,tBodyAcc-Mean-1,tBodyAcc-Mean-2,tBodyAcc-Mean-3,tBodyAcc-STD-1,tBodyAcc-STD-2,tBodyAcc-STD-3,tBodyAcc-Mad-1,tBodyAcc-Mad-2,tBodyAcc-Mad-3,tBodyAcc-Max-1,tBodyAcc-Max-2,tBodyAcc-Max-3,tBodyAcc-Min-1,tBodyAcc-Min-2,tBodyAcc-Min-3,tBodyAcc-SMA-1,tBodyAcc-Energy-1,tBodyAcc-Energy-2,tBodyAcc-Energy-3,tBodyAcc-IQR-1,tBodyAcc-IQR-2,tBodyAcc-IQR-3,tBodyAcc-ropy-1,tBodyAcc-ropy-2,tBodyAcc-ropy-3,tBodyAcc-ARCoeff-1,tBodyAcc-ARCoeff-2,tBodyAcc-ARCoeff-3,tBodyAcc-ARCoeff-4,tBodyAcc-ARCoeff-5,tBodyAcc-ARCoeff-6,tBodyAcc-ARCoeff-7,tBodyAcc-ARCoeff-8,tBodyAcc-ARCoeff-9,tBodyAcc-ARCoeff-10,tBodyAcc-ARCoeff-11,tBodyAcc-ARCoeff-12,tBodyAcc-Correlation-1,tBodyAcc-Correlation-2,tBodyAcc-Correlation-3,tGravityAcc-Mean-1,tGravityAcc-Mean-2,tGravityAcc-Mean-3,tGravityAcc-STD-1,tGravityAcc-STD-2,tGravityAcc-STD-3,tGravityAcc-Mad-1,tGravityAcc-Mad-2,tGravityAcc-Mad-3,tGravityAcc-Max-1,...,fBodyAccMag-MaxInds-1,fBodyAccMag-MeanFreq-1,fBodyAccMag-Skewness-1,fBodyAccMag-Kurtosis-1,fBodyAccJerkMag-Mean-1,fBodyAccJerkMag-STD-1,fBodyAccJerkMag-Mad-1,fBodyAccJerkMag-Max-1,fBodyAccJerkMag-Min-1,fBodyAccJerkMag-SMA-1,fBodyAccJerkMag-Energy-1,fBodyAccJerkMag-IQR-1,fBodyAccJerkMag-ropy-1,fBodyAccJerkMag-MaxInds-1,fBodyAccJerkMag-MeanFreq-1,fBodyAccJerkMag-Skewness-1,fBodyAccJerkMag-Kurtosis-1,fBodyGyroMag-Mean-1,fBodyGyroMag-STD-1,fBodyGyroMag-Mad-1,fBodyGyroMag-Max-1,fBodyGyroMag-Min-1,fBodyGyroMag-SMA-1,fBodyGyroMag-Energy-1,fBodyGyroMag-IQR-1,fBodyGyroMag-ropy-1,fBodyGyroMag-MaxInds-1,fBodyGyroMag-MeanFreq-1,fBodyGyroMag-Skewness-1,fBodyGyroMag-Kurtosis-1,fBodyGyroJerkMag-Mean-1,fBodyGyroJerkMag-STD-1,fBodyGyroJerkMag-Mad-1,fBodyGyroJerkMag-Max-1,fBodyGyroJerkMag-Min-1,fBodyGyroJerkMag-SMA-1,fBodyGyroJerkMag-Energy-1,fBodyGyroJerkMag-IQR-1,fBodyGyroJerkMag-ropy-1,fBodyGyroJerkMag-MaxInds-1,fBodyGyroJerkMag-MeanFreq-1,fBodyGyroJerkMag-Skewness-1,fBodyGyroJerkMag-Kurtosis-1,tBodyAcc-AngleWRTGravity-1,tBodyAccJerk-AngleWRTGravity-1,tBodyGyro-AngleWRTGravity-1,tBodyGyroJerk-AngleWRTGravity-1,tXAxisAcc-AngleWRTGravity-1,tYAxisAcc-AngleWRTGravity-1,tZAxisAcc-AngleWRTGravity-1
dtype,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,...,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64


,tBodyAcc-Mean-1,tBodyAcc-Mean-2,tBodyAcc-Mean-3,tBodyAcc-STD-1,tBodyAcc-STD-2,tBodyAcc-STD-3,tBodyAcc-Mad-1,tBodyAcc-Mad-2,tBodyAcc-Mad-3,tBodyAcc-Max-1,tBodyAcc-Max-2,tBodyAcc-Max-3,tBodyAcc-Min-1,tBodyAcc-Min-2,tBodyAcc-Min-3,tBodyAcc-SMA-1,tBodyAcc-Energy-1,tBodyAcc-Energy-2,tBodyAcc-Energy-3,tBodyAcc-IQR-1,tBodyAcc-IQR-2,tBodyAcc-IQR-3,tBodyAcc-ropy-1,tBodyAcc-ropy-2,tBodyAcc-ropy-3,tBodyAcc-ARCoeff-1,tBodyAcc-ARCoeff-2,tBodyAcc-ARCoeff-3,tBodyAcc-ARCoeff-4,tBodyAcc-ARCoeff-5,tBodyAcc-ARCoeff-6,tBodyAcc-ARCoeff-7,tBodyAcc-ARCoeff-8,tBodyAcc-ARCoeff-9,tBodyAcc-ARCoeff-10,tBodyAcc-ARCoeff-11,tBodyAcc-ARCoeff-12,tBodyAcc-Correlation-1,tBodyAcc-Correlation-2,tBodyAcc-Correlation-3,tGravityAcc-Mean-1,tGravityAcc-Mean-2,tGravityAcc-Mean-3,tGravityAcc-STD-1,tGravityAcc-STD-2,tGravityAcc-STD-3,tGravityAcc-Mad-1,tGravityAcc-Mad-2,tGravityAcc-Mad-3,tGravityAcc-Max-1,...,fBodyAccMag-MaxInds-1,fBodyAccMag-MeanFreq-1,fBodyAccMag-Skewness-1,fBodyAccMag-Kurtosis-1,fBodyAccJerkMag-Mean-1,fBodyAccJerkMag-STD-1,fBodyAccJerkMag-Mad-1,fBodyAccJerkMag-Max-1,fBodyAccJerkMag-Min-1,fBodyAccJerkMag-SMA-1,fBodyAccJerkMag-Energy-1,fBodyAccJerkMag-IQR-1,fBodyAccJerkMag-ropy-1,fBodyAccJerkMag-MaxInds-1,fBodyAccJerkMag-MeanFreq-1,fBodyAccJerkMag-Skewness-1,fBodyAccJerkMag-Kurtosis-1,fBodyGyroMag-Mean-1,fBodyGyroMag-STD-1,fBodyGyroMag-Mad-1,fBodyGyroMag-Max-1,fBodyGyroMag-Min-1,fBodyGyroMag-SMA-1,fBodyGyroMag-Energy-1,fBodyGyroMag-IQR-1,fBodyGyroMag-ropy-1,fBodyGyroMag-MaxInds-1,fBodyGyroMag-MeanFreq-1,fBodyGyroMag-Skewness-1,fBodyGyroMag-Kurtosis-1,fBodyGyroJerkMag-Mean-1,fBodyGyroJerkMag-STD-1,fBodyGyroJerkMag-Mad-1,fBodyGyroJerkMag-Max-1,fBodyGyroJerkMag-Min-1,fBodyGyroJerkMag-SMA-1,fBodyGyroJerkMag-Energy-1,fBodyGyroJerkMag-IQR-1,fBodyGyroJerkMag-ropy-1,fBodyGyroJerkMag-MaxInds-1,fBodyGyroJerkMag-MeanFreq-1,fBodyGyroJerkMag-Skewness-1,fBodyGyroJerkMag-Kurtosis-1,tBodyAcc-AngleWRTGravity-1,tBodyAccJerk-AngleWRTGravity-1,tBodyGyro-AngleWRTGravity-1,tBodyGyroJerk-AngleWRTGravity-1,tXAxisAcc-AngleWRTGravity-1,tYAxisAcc-AngleWRTGravity-1,tZAxisAcc-AngleWRTGravity-1
0,0.043580,-0.005970,-0.035054,-0.995381,-0.988366,-0.937382,-0.995007,-0.988816,-0.953325,-0.794796,-0.744893,-0.648447,0.841796,0.708440,0.651716,-0.975752,-0.999950,-0.999888,-0.998014,-0.993999,-0.991980,-0.970970,-0.547095,-0.700974,-0.622697,0.921884,-0.719483,0.342168,-0.161318,0.266049,-0.274351,0.267205,-0.020958,0.382610,-0.501748,0.512463,-0.206337,0.376778,0.435172,0.660199,0.960051,-0.135939,0.115556,-0.988134,-0.982693,-0.919723,-0.988362,-0.985523,-0.931834,0.892055,...,-0.842105,-0.061629,-0.446120,-0.797046,-0.993610,-0.994226,-0.992839,-0.993778,-0.988172,-0.993610,-0.999918,-0.991736,-1.0,-0.936508,0.349260,-0.517127,-0.801006,-0.980135,-0.961301,-0.974129,-0.956013,-0.989894,-0.980135,-0.999240,-0.992673,-0.701291,-1.000000,-0.132480,0.565697,0.363478,-0.991994,-0.990877,-0.990169,-0.992521,-0.991044,-0.991994,-0.999937,-0.990537,-0.871306,-1.000000,-0.012236,-0.314848,-0.713308,-0.112754,0.030400,-0.464761,-0.018446,-0.841559,0.179913,-0.051718
1,0.039480,-0.002131,-0.029067,-0.998348,-0.982945,-0.971273,-0.998702,-0.983315,-0.974000,-0.802537,-0.736338,-0.712415,0.838758,0.708440,0.659340,-0.987427,-0.999993,-0.999826,-0.999411,-0.998918,-0.985482,-0.973481,-0.781973,-0.534604,-0.593165,0.607435,-0.266783,0.275882,0.200417,0.131266,-0.149017,0.292436,-0.192986,0.217496,-0.089175,0.059909,-0.236609,-0.012696,-0.072711,0.578649,0.963215,-0.136648,0.109558,-0.997918,-0.990006,-0.955160,-0.998358,-0.990346,-0.956796,0.892060,...,-1.000000,-0.018270,-0.128777,-0.448744,-0.990220,-0.992431,-0.990594,-0.994902,-0.989545,-0.990220,-0.999867,-0.991506,-1.0,-0.841270,0.533688,-0.625993,-0.898311,-0.988296,-0.983313,-0.982951,-0.987406,-0.992134,-0.988296,-0.999811,-0.993996,-0.720683,-0.948718,-0.268979,-0.364219,-0.723724,-0.995857,-0.996580,-0.995671,-0.996939,-0.994436,-0.995857,-0.999981,-0.994623,-1.000000,-1.000000,0.202804,-0.603199,-0.860677,0.053477,-0.007435,-0.732626,0.

In [8]:
print('Size of test data', X_test.shape)
display(pd.DataFrame(X_test.dtypes, columns=['dtype']).T)
display(X_test.head(3))

Size of test data (3162, 561)


,tBodyAcc-Mean-1,tBodyAcc-Mean-2,tBodyAcc-Mean-3,tBodyAcc-STD-1,tBodyAcc-STD-2,tBodyAcc-STD-3,tBodyAcc-Mad-1,tBodyAcc-Mad-2,tBodyAcc-Mad-3,tBodyAcc-Max-1,tBodyAcc-Max-2,tBodyAcc-Max-3,tBodyAcc-Min-1,tBodyAcc-Min-2,tBodyAcc-Min-3,tBodyAcc-SMA-1,tBodyAcc-Energy-1,tBodyAcc-Energy-2,tBodyAcc-Energy-3,tBodyAcc-IQR-1,tBodyAcc-IQR-2,tBodyAcc-IQR-3,tBodyAcc-ropy-1,tBodyAcc-ropy-2,tBodyAcc-ropy-3,tBodyAcc-ARCoeff-1,tBodyAcc-ARCoeff-2,tBodyAcc-ARCoeff-3,tBodyAcc-ARCoeff-4,tBodyAcc-ARCoeff-5,tBodyAcc-ARCoeff-6,tBodyAcc-ARCoeff-7,tBodyAcc-ARCoeff-8,tBodyAcc-ARCoeff-9,tBodyAcc-ARCoeff-10,tBodyAcc-ARCoeff-11,tBodyAcc-ARCoeff-12,tBodyAcc-Correlation-1,tBodyAcc-Correlation-2,tBodyAcc-Correlation-3,tGravityAcc-Mean-1,tGravityAcc-Mean-2,tGravityAcc-Mean-3,tGravityAcc-STD-1,tGravityAcc-STD-2,tGravityAcc-STD-3,tGravityAcc-Mad-1,tGravityAcc-Mad-2,tGravityAcc-Mad-3,tGravityAcc-Max-1,...,fBodyAccMag-MaxInds-1,fBodyAccMag-MeanFreq-1,fBodyAccMag-Skewness-1,fBodyAccMag-Kurtosis-1,fBodyAccJerkMag-Mean-1,fBodyAccJerkMag-STD-1,fBodyAccJerkMag-Mad-1,fBodyAccJerkMag-Max-1,fBodyAccJerkMag-Min-1,fBodyAccJerkMag-SMA-1,fBodyAccJerkMag-Energy-1,fBodyAccJerkMag-IQR-1,fBodyAccJerkMag-ropy-1,fBodyAccJerkMag-MaxInds-1,fBodyAccJerkMag-MeanFreq-1,fBodyAccJerkMag-Skewness-1,fBodyAccJerkMag-Kurtosis-1,fBodyGyroMag-Mean-1,fBodyGyroMag-STD-1,fBodyGyroMag-Mad-1,fBodyGyroMag-Max-1,fBodyGyroMag-Min-1,fBodyGyroMag-SMA-1,fBodyGyroMag-Energy-1,fBodyGyroMag-IQR-1,fBodyGyroMag-ropy-1,fBodyGyroMag-MaxInds-1,fBodyGyroMag-MeanFreq-1,fBodyGyroMag-Skewness-1,fBodyGyroMag-Kurtosis-1,fBodyGyroJerkMag-Mean-1,fBodyGyroJerkMag-STD-1,fBodyGyroJerkMag-Mad-1,fBodyGyroJerkMag-Max-1,fBodyGyroJerkMag-Min-1,fBodyGyroJerkMag-SMA-1,fBodyGyroJerkMag-Energy-1,fBodyGyroJerkMag-IQR-1,fBodyGyroJerkMag-ropy-1,fBodyGyroJerkMag-MaxInds-1,fBodyGyroJerkMag-MeanFreq-1,fBodyGyroJerkMag-Skewness-1,fBodyGyroJerkMag-Kurtosis-1,tBodyAcc-AngleWRTGravity-1,tBodyAccJerk-AngleWRTGravity-1,tBodyGyro-AngleWRTGravity-1,tBodyGyroJerk-AngleWRTGravity-1,tXAxisAcc-AngleWRTGravity-1,tYAxisAcc-AngleWRTGravity-1,tZAxisAcc-AngleWRTGravity-1
dtype,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,...,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64


,tBodyAcc-Mean-1,tBodyAcc-Mean-2,tBodyAcc-Mean-3,tBodyAcc-STD-1,tBodyAcc-STD-2,tBodyAcc-STD-3,tBodyAcc-Mad-1,tBodyAcc-Mad-2,tBodyAcc-Mad-3,tBodyAcc-Max-1,tBodyAcc-Max-2,tBodyAcc-Max-3,tBodyAcc-Min-1,tBodyAcc-Min-2,tBodyAcc-Min-3,tBodyAcc-SMA-1,tBodyAcc-Energy-1,tBodyAcc-Energy-2,tBodyAcc-Energy-3,tBodyAcc-IQR-1,tBodyAcc-IQR-2,tBodyAcc-IQR-3,tBodyAcc-ropy-1,tBodyAcc-ropy-2,tBodyAcc-ropy-3,tBodyAcc-ARCoeff-1,tBodyAcc-ARCoeff-2,tBodyAcc-ARCoeff-3,tBodyAcc-ARCoeff-4,tBodyAcc-ARCoeff-5,tBodyAcc-ARCoeff-6,tBodyAcc-ARCoeff-7,tBodyAcc-ARCoeff-8,tBodyAcc-ARCoeff-9,tBodyAcc-ARCoeff-10,tBodyAcc-ARCoeff-11,tBodyAcc-ARCoeff-12,tBodyAcc-Correlation-1,tBodyAcc-Correlation-2,tBodyAcc-Correlation-3,tGravityAcc-Mean-1,tGravityAcc-Mean-2,tGravityAcc-Mean-3,tGravityAcc-STD-1,tGravityAcc-STD-2,tGravityAcc-STD-3,tGravityAcc-Mad-1,tGravityAcc-Mad-2,tGravityAcc-Mad-3,tGravityAcc-Max-1,...,fBodyAccMag-MaxInds-1,fBodyAccMag-MeanFreq-1,fBodyAccMag-Skewness-1,fBodyAccMag-Kurtosis-1,fBodyAccJerkMag-Mean-1,fBodyAccJerkMag-STD-1,fBodyAccJerkMag-Mad-1,fBodyAccJerkMag-Max-1,fBodyAccJerkMag-Min-1,fBodyAccJerkMag-SMA-1,fBodyAccJerkMag-Energy-1,fBodyAccJerkMag-IQR-1,fBodyAccJerkMag-ropy-1,fBodyAccJerkMag-MaxInds-1,fBodyAccJerkMag-MeanFreq-1,fBodyAccJerkMag-Skewness-1,fBodyAccJerkMag-Kurtosis-1,fBodyGyroMag-Mean-1,fBodyGyroMag-STD-1,fBodyGyroMag-Mad-1,fBodyGyroMag-Max-1,fBodyGyroMag-Min-1,fBodyGyroMag-SMA-1,fBodyGyroMag-Energy-1,fBodyGyroMag-IQR-1,fBodyGyroMag-ropy-1,fBodyGyroMag-MaxInds-1,fBodyGyroMag-MeanFreq-1,fBodyGyroMag-Skewness-1,fBodyGyroMag-Kurtosis-1,fBodyGyroJerkMag-Mean-1,fBodyGyroJerkMag-STD-1,fBodyGyroJerkMag-Mad-1,fBodyGyroJerkMag-Max-1,fBodyGyroJerkMag-Min-1,fBodyGyroJerkMag-SMA-1,fBodyGyroJerkMag-Energy-1,fBodyGyroJerkMag-IQR-1,fBodyGyroJerkMag-ropy-1,fBodyGyroJerkMag-MaxInds-1,fBodyGyroJerkMag-MeanFreq-1,fBodyGyroJerkMag-Skewness-1,fBodyGyroJerkMag-Kurtosis-1,tBodyAcc-AngleWRTGravity-1,tBodyAccJerk-AngleWRTGravity-1,tBodyGyro-AngleWRTGravity-1,tBodyGyroJerk-AngleWRTGravity-1,tXAxisAcc-AngleWRTGravity-1,tYAxisAcc-AngleWRTGravity-1,tZAxisAcc-AngleWRTGravity-1
0,0.030914,-0.008927,0.040382,-0.938504,-0.944626,-0.759334,-0.952398,-0.950281,-0.802483,-0.757099,-0.733398,-0.407960,0.728511,0.658266,0.629169,-0.913306,-0.997966,-0.998683,-0.975818,-0.975440,-0.966793,-0.914396,-0.363822,-0.420897,0.181772,0.517928,-0.403636,0.449325,-0.147545,0.223150,-0.190143,0.155899,0.054035,0.053156,-0.000037,-0.077888,0.028585,0.077659,-0.490616,-0.711964,0.933151,-0.277360,0.115469,-0.940012,-0.940460,-0.714148,-0.945699,-0.942284,-0.744700,0.906083,...,-1.000000,-0.448379,0.005537,-0.383555,-0.894952,-0.896808,-0.889015,-0.928866,-0.898096,-0.894952,-0.993471,-0.921836,-0.484619,-1.0,-0.031754,-0.255114,-0.698671,-0.770610,-0.797105,-0.769223,-0.834266,-0.940359,-0.770610,-0.970958,-0.798403,0.179435,-1.000000,-0.054576,-0.496820,-0.764755,-0.890169,-0.907480,-0.895518,-0.917953,-0.909825,-0.890169,-0.994105,-0.898097,-0.234815,-1.000000,0.122830,-0.345684,-0.709087,0.006462,0.162920,-0.825886,0.271151,-0.720559,0.276779,-0.051074
1,0.042548,0.001079,-0.026236,-0.975516,-0.977502,-0.960146,-0.986694,-0.978983,-0.966820,-0.757099,-0.733398,-0.701699,0.770927,0.706301,0.635732,-0.978295,-0.999618,-0.999708,-0.999158,-0.993775,-0.982664,-0.974733,-0.466563,-0.392219,-0.435565,0.504355,-0.117122,0.062621,0.188184,0.209414,-0.245994,0.390254,-0.247800,0.210276,-0.146806,0.106145,-0.022423,-0.104187,-0.429196,0.398133,0.924068,-0.283835,0.152755,-0.991197,-0.984752,-0.976904,-0.991464,-0.984357,-0.977489,0.856176,...,-0.894737,0.224164,-0.214108,-0.599074,-0.945325,-0.934609,-0.930151,-0.950377,-0.967702,-0.945325,-0.997610,-0.957543,-0.723642,-1.0,-0.486025,-0.216243,-0.664383,-0.924461,-0.916766,-0.920964,-0.924413,-0.991454,-0.924461,-0.995727,-0.933145,-0.218860,-1.000000,-0.039379,-0.116580,-0.467260,-0.951981,-0.938387,-0.938230,-0.949994,-0.986899,-0.951981,-0.998272,-0.951332,-0.431053,-1.000000,-0.314688,-0.142804,-0.600867,-0.083495,0.017500,-0.434375,0.920

In [9]:
activity_labels = pd.read_table(root + 'activity_labels.txt', header=None).values.flatten()
activity_labels = np.array([label.rstrip().split() for label in activity_labels])
label2activity_dict = {}
activity2label_dict = {}
for label, activity in activity_labels:
    label2activity_dict[int(label)] = activity
    activity2label_dict[activity] = int(label)

In [23]:
classe_names_inc = ['WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'SITTING', 'STANDING', 'LAYING']
classe_ids_inc = [activity2label_dict[c] for c in classe_names_inc]

In [33]:
y_train[y_train[0].isin(classe_ids_inc)].index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            7757, 7758, 7759, 7760, 7761, 7762, 7763, 7764, 7765, 7766],
           dtype='int64', length=7415)

In [37]:
X_train.iloc[y_train[y_train[0].isin(classe_ids_inc)].index].reset_index()

,index,tBodyAcc-Mean-1,tBodyAcc-Mean-2,tBodyAcc-Mean-3,tBodyAcc-STD-1,tBodyAcc-STD-2,tBodyAcc-STD-3,tBodyAcc-Mad-1,tBodyAcc-Mad-2,tBodyAcc-Mad-3,tBodyAcc-Max-1,tBodyAcc-Max-2,tBodyAcc-Max-3,tBodyAcc-Min-1,tBodyAcc-Min-2,tBodyAcc-Min-3,tBodyAcc-SMA-1,tBodyAcc-Energy-1,tBodyAcc-Energy-2,tBodyAcc-Energy-3,tBodyAcc-IQR-1,tBodyAcc-IQR-2,tBodyAcc-IQR-3,tBodyAcc-ropy-1,tBodyAcc-ropy-2,tBodyAcc-ropy-3,tBodyAcc-ARCoeff-1,tBodyAcc-ARCoeff-2,tBodyAcc-ARCoeff-3,tBodyAcc-ARCoeff-4,tBodyAcc-ARCoeff-5,tBodyAcc-ARCoeff-6,tBodyAcc-ARCoeff-7,tBodyAcc-ARCoeff-8,tBodyAcc-ARCoeff-9,tBodyAcc-ARCoeff-10,tBodyAcc-ARCoeff-11,tBodyAcc-ARCoeff-12,tBodyAcc-Correlation-1,tBodyAcc-Correlation-2,tBodyAcc-Correlation-3,tGravityAcc-Mean-1,tGravityAcc-Mean-2,tGravityAcc-Mean-3,tGravityAcc-STD-1,tGravityAcc-STD-2,tGravityAcc-STD-3,tGravityAcc-Mad-1,tGravityAcc-Mad-2,tGravityAcc-Mad-3,...,fBodyAccMag-MaxInds-1,fBodyAccMag-MeanFreq-1,fBodyAccMag-Skewness-1,fBodyAccMag-Kurtosis-1,fBodyAccJerkMag-Mean-1,fBodyAccJerkMag-STD-1,fBodyAccJerkMag-Mad-1,fBodyAccJerkMag-Max-1,fBodyAccJerkMag-Min-1,fBodyAccJerkMag-SMA-1,fBodyAccJerkMag-Energy-1,fBodyAccJerkMag-IQR-1,fBodyAccJerkMag-ropy-1,fBodyAccJerkMag-MaxInds-1,fBodyAccJerkMag-MeanFreq-1,fBodyAccJerkMag-Skewness-1,fBodyAccJerkMag-Kurtosis-1,fBodyGyroMag-Mean-1,fBodyGyroMag-STD-1,fBodyGyroMag-Mad-1,fBodyGyroMag-Max-1,fBodyGyroMag-Min-1,fBodyGyroMag-SMA-1,fBodyGyroMag-Energy-1,fBodyGyroMag-IQR-1,fBodyGyroMag-ropy-1,fBodyGyroMag-MaxInds-1,fBodyGyroMag-MeanFreq-1,fBodyGyroMag-Skewness-1,fBodyGyroMag-Kurtosis-1,fBodyGyroJerkMag-Mean-1,fBodyGyroJerkMag-STD-1,fBodyGyroJerkMag-Mad-1,fBodyGyroJerkMag-Max-1,fBodyGyroJerkMag-Min-1,fBodyGyroJerkMag-SMA-1,fBodyGyroJerkMag-Energy-1,fBodyGyroJerkMag-IQR-1,fBodyGyroJerkMag-ropy-1,fBodyGyroJerkMag-MaxInds-1,fBodyGyroJerkMag-MeanFreq-1,fBodyGyroJerkMag-Skewness-1,fBodyGyroJerkMag-Kurtosis-1,tBodyAcc-AngleWRTGravity-1,tBodyAccJerk-AngleWRTGravity-1,tBodyGyro-AngleWRTGravity-1,tBodyGyroJerk-AngleWRTGravity-1,tXAxisAcc-AngleWRTGravity-1,tYAxisAcc-AngleWRTGravity-1,tZAxisAcc-AngleWRTGravity-1
0,0,0.043580,-0.005970,-0.035054,-0.995381,-0.988366,-0.937382,-0.995007,-0.988816,-0.953325,-0.794796,-0.744893,-0.648447,0.841796,0.708440,0.651716,-0.975752,-0.999950,-0.999888,-0.998014,-0.993999,-0.991980,-0.970970,-0.547095,-0.700974,-0.622697,0.921884,-0.719483,0.342168,-0.161318,0.266049,-0.274351,0.267205,-0.020958,0.382610,-0.501748,0.512463,-0.206337,0.376778,0.435172,0.660199,0.960051,-0.135939,0.115556,-0.988134,-0.982693,-0.919723,-0.988362,-0.985523,-0.931834,...,-0.842105,-0.061629,-0.446120,-0.797046,-0.993610,-0.994226,-0.992839,-0.993778,-0.988172,-0.993610,-0.999918,-0.991736,-1.000000,-0.936508,0.349260,-0.517127,-0.801006,-0.980135,-0.961301,-0.974129,-0.956013,-0.989894,-0.980135,-0.999240,-0.992673,-0.701291,-1.000000,-0.132480,0.565697,0.363478,-0.991994,-0.990877,-0.990169,-0.992521,-0.991044,-0.991994,-0.999937,-0.990537,-0.871306,-1.000000,-0.012236,-0.314848,-0.713308,-0.112754,0.030400,-0.464761,-0.018446,-0.841559,0.179913,-0.051718
1,1,0.039480,-0.002131,-0.029067,-0.998348,-0.982945,-0.971273,-0.998702,-0.983315,-0.974000,-0.802537,-0.736338,-0.712415,0.838758,0.708440,0.659340,-0.987427,-0.999993,-0.999826,-0.999411,-0.998918,-0.985482,-0.973481,-0.781973,-0.534604,-0.593165,0.607435,-0.266783,0.275882,0.200417,0.131266,-0.149017,0.292436,-0.192986,0.217496,-0.089175,0.059909,-0.236609,-0.012696,-0.072711,0.578649,0.963215,-0.136648,0.109558,-0.997918,-0.990006,-0.955160,-0.998358,-0.990346,-0.956796,...,-1.000000,-0.018270,-0.128777,-0.448744,-0.990220,-0.992431,-0.990594,-0.994902,-0.989545,-0.990220,-0.999867,-0.991506,-1.000000,-0.841270,0.533688,-0.625993,-0.898311,-0.988296,-0.983313,-0.982951,-0.987406,-0.992134,-0.988296,-0.999811,-0.993996,-0.720683,-0.948718,-0.268979,-0.364219,-0.723724,-0.995857,-0.996580,-0.995671,-0.996939,-0.994436,-0.995857,-0.999981,-0.994623,-1.000000,-1.000000,0.202804,-0.603199,-0.860677,0.053477,-0.007435,-0.732626,0.703511,-0.845092

In [35]:
y_train.iloc[y_train[y_train[0].isin(classe_ids_inc)].index]

,0
0,5
1,5
2,5
3,5
4,5
...,...
7762,2
7763,2
7764,2
7765,2


In [18]:
print(list(activity2label_dict.keys()))

['WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'SITTING', 'STANDING', 'LAYING', 'STAND_TO_SIT', 'SIT_TO_STAND', 'SIT_TO_LIE', 'LIE_TO_SIT', 'STAND_TO_LIE', 'LIE_TO_STAND']


In [10]:
print('Size of train label', y_train.shape)
c = Counter(y_train.values.flatten())
for label in range(1, len(c)+1):
    print(f'label {label} ({label2activity_dict[label]}) has {c[label]} samples ({c[label] / len(y_train) * 100:.04}% of train data)')

Size of train label (7767, 1)
label 1 (WALKING) has 1226 samples (15.78% of train data)
label 2 (WALKING_UPSTAIRS) has 1073 samples (13.81% of train data)
label 3 (WALKING_DOWNSTAIRS) has 987 samples (12.71% of train data)
label 4 (SITTING) has 1293 samples (16.65% of train data)
label 5 (STANDING) has 1423 samples (18.32% of train data)
label 6 (LAYING) has 1413 samples (18.19% of train data)
label 7 (STAND_TO_SIT) has 47 samples (0.6051% of train data)
label 8 (SIT_TO_STAND) has 23 samples (0.2961% of train data)
label 9 (SIT_TO_LIE) has 75 samples (0.9656% of train data)
label 10 (LIE_TO_SIT) has 60 samples (0.7725% of train data)
label 11 (STAND_TO_LIE) has 90 samples (1.159% of train data)
label 12 (LIE_TO_STAND) has 57 samples (0.7339% of train data)


In [11]:
print('Size of test label', y_test.shape)
c = Counter(y_test.values.flatten())
for label in range(1, len(c)+1):
    print(f'label {label} ({label2activity_dict[label]}) has {c[label]} samples ({c[label] / len(y_test) * 100:.04}% of test data)')

Size of test label (3162, 1)
label 1 (WALKING) has 496 samples (15.69% of test data)
label 2 (WALKING_UPSTAIRS) has 471 samples (14.9% of test data)
label 3 (WALKING_DOWNSTAIRS) has 420 samples (13.28% of test data)
label 4 (SITTING) has 508 samples (16.07% of test data)
label 5 (STANDING) has 556 samples (17.58% of test data)
label 6 (LAYING) has 545 samples (17.24% of test data)
label 7 (STAND_TO_SIT) has 23 samples (0.7274% of test data)
label 8 (SIT_TO_STAND) has 10 samples (0.3163% of test data)
label 9 (SIT_TO_LIE) has 32 samples (1.012% of test data)
label 10 (LIE_TO_SIT) has 25 samples (0.7906% of test data)
label 11 (STAND_TO_LIE) has 49 samples (1.55% of test data)
label 12 (LIE_TO_STAND) has 27 samples (0.8539% of test data)


In [12]:
subject_id_train = pd.read_table(root + 'Train/subject_id_train.txt', header=None).values.flatten()
subject_id_test = pd.read_table(root + 'Test/subject_id_test.txt', header=None).values.flatten()

print('subjects in train are: ', set(subject_id_train))
print('subjects in test are: ', set(subject_id_test))

subjects in train are:  {1, 3, 5, 6, 7, 8, 11, 14, 15, 16, 17, 19, 21, 22, 23, 25, 26, 27, 28, 29, 30}
subjects in test are:  {2, 4, 9, 10, 12, 13, 18, 20, 24}
